# Hand Written digit recognition using CNN
In this notebook, I have converted dataset of MNIST from csv format to binary images so that we can visualize and can solve it using Convolutional Neural Networks(CNN).
with the available csv format dataset, this problem can surely be solved with other algorithms in better way. but my main pupose to write this notebook is to give insights of using CNN in keras with simpler dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, ImageDataGenerator
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



### In the cell below, we are reading dataset of hand written digits in pandas dataframe. because data was available in csv format

In [ ]:
train_data = pd.read_csv ('/kaggle/input/digit-recognizer/train.csv')
train_data.head()

### From CSV data to Images
In cell below, I wrote some code to preprocess the data according to my need. I get dataframe of images where each row contains 784 pixel values of one image. I took each row and converted it to 28x28 images and stored all images in a list.
from given data, I also extracted lables that is calssifying each image.

In [ ]:
from keras.utils import to_categorical

def extract_train_images(data):
    """
    function to extract list 2d numpy arrays of images and numpy array of labels.
    Input - DataFrame data
    Output - list of 2D numpy arrays for images and 1d numpy array for labels
    
    """
    labels = np.array(data['label'])
    
    data = data.drop('label', axis = 'columns')
    images_array = np.array(data)
    
    images_list = []
    for image_arr in images_array:
        images_list.append(np.array(np.array_split(image_arr, 28))[:,:,np.newaxis])
    
    return images_list, labels

images_list, labels = extract_train_images(train_data)
one_hot_labels = to_categorical(labels)
print(labels)

## Let's plot some images as well as print their labels to visualize the data

In [ ]:
count = 1
fig = plt.figure(figsize = (8,8))
rows = 4
columns = 4
for image, label in zip(images_list, labels):
    fig.add_subplot(rows, columns, count)
    plt.imshow(image[:,:,0])
    print('Label:',label)
    count+=1
    if count >16:
        break
fig.show()

## In cell below, I am using Image data generator available in keras. you can check here in documentation [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
#Training Data Generation
train_gen = ImageDataGenerator(rescale = 1/255.)

train_images = np.array(images_list)
print(train_images.shape)

train_generator = train_gen.flow(train_images, one_hot_labels, batch_size = 32)

### Enough preprocessing with training data now, let's import test data and play with it so that we can prepare it to make predictions that can be submitted later.

In [ ]:
#Test/Validation dataset

test_data = pd.read_csv ('/kaggle/input/digit-recognizer/test.csv')
test_data.head()

In [ ]:
def extract_test_images(data):
    images_array = np.array(data)
    
    images_list = []
    for image_arr in images_array:
        images_list.append(np.array(np.array_split(image_arr, 28))[:,:,np.newaxis])
    
    return images_list

test_images_list = extract_test_images(test_data)

test_images = np.array(test_images_list)
print(test_images.shape)

## In the cell below, I have defined model using keras API layers. It is a simple CNN architecture i haven't used any complex state of the art architecture to make it simple.

In [ ]:
# Model Definition

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

model = tf.keras.Sequential([
    Conv2D(32, kernel_size = (3,3), input_shape = (28,28,1), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(64, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(28, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Here I will define callback function that will be called after every epoch on training dataset. I am forcing an early stopping if accuracy will reach 100% using this callback

In [ ]:
import keras

class myCallBack(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        acc = logs.get('accuracy')
        if acc >= 1.0:
            print("\nRequired accuracy achieved so ending the training.")
            self.model.stop_training=True

call_back = myCallBack()

### Training of model on 100 epochs

In [ ]:
BS = 32
EPOCHS = 10

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_images.shape[0] // BS,
                              epochs=EPOCHS,
                              callbacks = [call_back])

In [ ]:
model.save('digit_recognizer.h5')

In [ ]:
model.summary()

### As model is using softmax activation at the end so it will output a vector. Let's convert that one-hot vector to label so that we can print and see predictions by our model.

In [ ]:
def onehot_to_label(vec):
    """
    function will convert one hot vector of model output into a single label
    """
    count=0
    for number in vec[0]:
        if number == 1:
            return count
        count+=1
    return -1

### Visualization of images and their predictions made by our trained model.

In [ ]:
count = 1
fig = plt.figure(figsize = (8,8))
rows = 4
columns = 4
for image in test_images:
    fig.add_subplot(rows, columns, count)
    plt.imshow(image[:,:,0])
    print(onehot_to_label(model.predict(image.reshape(1,28,28,1))))
    count+=1
    if count >16:
        break
fig.show()

### If you liked the notebook kindly support it by giving a vote. Thank you.